In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [32]:
# This is a script to flatten and save all the relevant layer weights as binary files
# KEY IDEA - be consistent with filenames

import numpy as np
import os

import tensorflow as tf

os.chdir('/content/drive/MyDrive/EE113DB - Sagar')
print(os.getcwd())

/content/drive/MyDrive/EE113DB - Sagar


In [ ]:
layer_dir = 'layer_weights'

model = tf.keras.models.load_model('set_yolov3-tiny.h5')
model.summary()

In [34]:
layer_list = model.layers

layer_names_with_parameters = ['conv2d', 'batch_normalization', 
                               'conv2d_1', 'batch_normalization_1',
                               'conv2d_2', 'batch_normalization_2',
                               'conv2d_3', 'batch_normalization_3',
                               'conv2d_4', 'batch_normalization_4',
                               'conv2d_5', 'batch_normalization_5',
                               'conv2d_6', 'batch_normalization_6',
                               'conv2d_7', 'batch_normalization_7',
                               'conv2d_10',
                               'batch_normalization_9',
                               'conv2d_8',
                               'conv2d_11',
                               'batch_normalization_8',
                               'batch_normalization_10',
                               'conv2d_9',
                               'conv2d_12'
                               ]
# Remark: Only conv2d_9 and conv2d_12 have biases 

In [ ]:
layer = model.get_layer('conv2d_12')
for property, value in vars(layer).items():
    print(property, ":", value)

In [ ]:
print(layer.get_weights())

In [40]:
for layer_name in layer_names_with_parameters:
  layer = model.get_layer(name=layer_name)
  l = layer_name.split("_")
  base_filename = os.path.join(layer_dir, layer_name)

  if l[0] == 'conv2d':
    # Save both weights '.w' and biases '.b'
    if layer_name == 'conv2d_9' or layer_name == 'conv2d_12':
      w, b = layer.get_weights()[0], layer.get_weights()[1]
      b = np.float32(b)
      w = np.float32(w.flatten())

      w.tofile(base_filename + '.w')
      b.tofile(base_filename + '.b')
    # Save just the weights '.w' - in C file treat the bias at zero or add case for this in Conv2D function
    else:
      w = np.float32(layer.get_weights()[0])

      w.tofile(base_filename + '.w')
  else:
    bn_moving_mean = np.float32((layer.moving_mean).numpy())
    bn_moving_variance = np.float32((layer.moving_variance).numpy())
    bn_gamma = np.float32((layer.gamma).numpy())
    bn_beta = np.float32((layer.beta).numpy())

    bn_moving_mean.tofile(base_filename + '.moving_mean')
    bn_moving_variance.tofile(base_filename + '.moving_variance')
    bn_gamma.tofile(base_filename + '.gamma')
    bn_beta.tofile(base_filename + '.beta')